In [3]:
def Preprocess(adata_ref,output_path,labels_key):
    import sys
    IN_COLAB = "google.colab" in sys.modules
    if IN_COLAB and branch == "stable":
        !pip install --quiet git+https://github.com/BayraktarLab/cell2location#egg=cell2location[tutorials]
    import scanpy as sc
    import anndata
    import pandas as pd
    import numpy as np
    import matplotlib.pyplot as plt
    import matplotlib as mpl
    import cell2location
    import scvi
    from matplotlib import rcParams
    rcParams['pdf.fonttype'] = 42 # enables correct plotting of text
    import seaborn as sns
    import loompy
    import pyreadr
    import os
    # Use ENSEMBL as gene IDs to make sure IDs are unique and correctly matched
    adata_ref.var['SYMBOL']=adata_ref.var.index
    adata_ref.var.index = adata_ref.var['GeneID-2'].copy()
    adata_ref.var_names = adata_ref.var['GeneID-2'].copy()
    adata_ref.var.index.name = None
    # adata_ref.raw.var['SYMBOL'] = adata_ref.raw.var.index
    # adata_ref.raw.var.index = adata_ref.raw.var['GeneID-2'].copy()
    # adata_ref.raw.var.index.name = None
    # before we estimate the reference cell type signature we recommend to perform very permissive genes selection
    # in this 2D histogram orange rectangle lays over excluded genes.
    # In this case, the downloaded dataset was already filtered using this method,
    # hence no density under the orange rectangle
    from cell2location.utils.filtering import filter_genes
    selected = filter_genes(adata_ref, cell_count_cutoff=5, cell_percentage_cutoff2=0.03, nonz_mean_cutoff=1.12)
    # filter the object
    adata_ref = adata_ref[:, selected].copy()
    print(111)
    
    cell2location.models.RegressionModel.setup_anndata(adata=adata_ref,
                                                           # 10X reaction / sample / batch
                                                           batch_key='orig.ident',               
                                                           # cell type, covariate used for constructing signatures
                                                           labels_key=labels_key,
                                                           # multiplicative technical effects (platform, 3' vs 5', donor effect)
                                                           categorical_covariate_keys=['orig.ident']
                                                          )
    print(222)
    
    # create the regression model
    from cell2location.models import RegressionModel
    mod = RegressionModel(adata_ref)
    # view anndata_setup as a sanity check
    mod.view_anndata_setup()
    # create and train the regression model
    from cell2location.models import RegressionModel
    mod = RegressionModel(adata_ref)
    # Use all data for training (validation not implemented yet, train_size=1)
    mod.train(max_epochs=1000, use_gpu=True)
    # plot ELBO loss history during training, removing first 20 epochs from the plot
    mod.plot_history(20)
    # In this section, we export the estimated cell abundance (summary of the posterior distribution).
    adata_ref = mod.export_posterior(
        adata_ref, sample_kwargs={'num_samples': 1000, 'batch_size': 2500, 'use_gpu': True}
    )
    # Save model
    mod.save(f"{output_path}{labels_key}_model_sc.pt", overwrite=True)
    # Save anndata object with results
    adata_file = f"{output_path}sc_{labels_key}.h5ad"
    adata_ref.write(adata_file)
    return(adata_ref)
